Ad click prediction
===
insert intro

Cleansing and Inspection
---
First, let's read the relevant columns into memory.

In [3]:
from pandas import read_csv
ads = read_csv('data/ads.csv', usecols=['click', 'AdvertiserID', 'AdExchange', 'Adslotwidth',
                                           'Adslotheight', 'Adslotvisibility', 'Adslotformat', 'Biddingprice', 'Browser', 'imp', 'interest_news',
                                           'interest_eduation', 'interest_automobile', 'interest_realestate',
                                           'interest_IT', 'interest_electronicgame', 'interest_fashion',
                                           'interest_entertainment', 'interest_luxury', 'interest_homeandlifestyle',
                                           'interest_health', 'interest_food', 'interest_divine',
                                           'interest_motherhood_parenting', 'interest_sports',
                                           'interest_travel_outdoors',
                                           'interest_social', 'interest_art_photography_design',
                                           'interest_onlineliterature', 'interest_3c', 'interest_culture',
                                           'interest_sex',
                                           'Inmarket_3cproduct', 'Inmarket_appliances', 'Inmarket_clothing_shoes_bags',
                                           'Inmarket_Beauty_PersonalCare', 'Inmarket_infant_momproducts',
                                           'Inmarket_sportsitem', 'Inmarket_outdoor', 'Inmarket_healthcareproducts',
                                           'Inmarket_luxury', 'Inmarket_realestate', 'Inmarket_automobile',
                                           'Inmarket_finance', 'Inmarket_travel', 'Inmarket_education',
                                           'Inmarket_service', 'Inmarket_electronicgame', 'Inmarket_book',
                                           'Inmarket_medicine', 'Inmarket_food_drink', 'Inmarket_homeimprovement',
                                           'Demographic_gender_male', 'Demographic_gender_famale', 'Payingprice'])
if 'Unnamed: 0' in ads:
    ads.drop('Unnamed: 0', axis=1, inplace=True)

~/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (4,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Let's inspect the columns of the dataset.

In [5]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4621497 entries, 0 to 4621496
Data columns (total 55 columns):
Browser                            object
AdExchange                         float64
Adslotwidth                        float64
Adslotheight                       float64
Adslotvisibility                   object
Adslotformat                       object
Biddingprice                       float64
AdvertiserID                       float64
imp                                float64
click                              float64
interest_news                      float64
interest_eduation                  float64
interest_automobile                float64
interest_realestate                float64
interest_IT                        float64
interest_electronicgame            float64
interest_fashion                   float64
interest_entertainment             float64
interest_luxury                    float64
interest_homeandlifestyle          float64
interest_health               

We see that we have ~ 4 million observations and 55 variables. The most important variable is the `click` variable, which indicates whether the ad was clicked on by the user. Let's see how many missing values we have.

In [6]:
ads['click'].isnull().sum()

1182166

Around 1 million bids do not contain a value for the `click` variable. However, this variable is critical, because we will regress it later on. Therefore, we need to remove these observations.

In [8]:
ads.dropna(subset=['click'], inplace=True)

Additionaly, let's move the `click` variable to the front.

In [10]:
cols = ['click'] + [col for col in ads if col != 'click']
ads = ads[cols]

Let's fix some typos.

In [11]:
ads.rename(
    columns={'interest_eduation': 'interest_education', 'Demographic_gender_famale': 'Demographic_gender_female'},
    inplace=True)

We see that most variables are detected as floating point numbers. However, most of them are actually boolean variables! We need to account for that and convert them. First, let's look at the average missing values per variable.

In [12]:
boolean_cols = ['imp', 'click', 'interest_news',
                'interest_education', 'interest_automobile', 'interest_realestate',
                'interest_IT', 'interest_electronicgame', 'interest_fashion',
                'interest_entertainment', 'interest_luxury', 'interest_homeandlifestyle',
                'interest_health', 'interest_food', 'interest_divine',
                'interest_motherhood_parenting', 'interest_sports', 'interest_travel_outdoors',
                'interest_social', 'interest_art_photography_design',
                'interest_onlineliterature', 'interest_3c', 'interest_culture', 'interest_sex',
                'Inmarket_3cproduct', 'Inmarket_appliances', 'Inmarket_clothing_shoes_bags',
                'Inmarket_Beauty_PersonalCare', 'Inmarket_infant_momproducts',
                'Inmarket_sportsitem', 'Inmarket_outdoor', 'Inmarket_healthcareproducts',
                'Inmarket_luxury', 'Inmarket_realestate', 'Inmarket_automobile',
                'Inmarket_finance', 'Inmarket_travel', 'Inmarket_education',
                'Inmarket_service', 'Inmarket_electronicgame', 'Inmarket_book',
                'Inmarket_medicine', 'Inmarket_food_drink', 'Inmarket_homeimprovement',
                'Demographic_gender_male', 'Demographic_gender_female']
missings = 0
for col in boolean_cols:
    missings_col = ads[col].isnull().sum()
    if col == 'imp':
        print('Missing values in `imp`', missings_col)
    missings += ads[col].isnull().sum()
missings / len(boolean_cols)



Missing values in `imp` 0


1409622.2608695652

These are quite a lot, and we cannot afford to loose them. As the variables are simply dummy variables of a categorical variable, we decide to simply use the default bool conversion strategy in this case, ie convert all missing values to `False`. Intuitively, if we don't know whether a user is interested in food, we simply assume that the user isn't interested in it. This way, we won't skew the analysis significantly. Further note, that the `imp` variable has no missing values. 

In [14]:
ads[boolean_cols] = ads[boolean_cols].fillna(0)
ads[boolean_cols] = ads[boolean_cols].astype(bool)

How many bids did not win impressions?

In [15]:
(ads['imp'] == False).sum()

1473696

Obviously, if the advertiser didn't win the impression, the user won't be able to click. Therefore, we use only the won impressions and remove the column. As an aside, it would be an interesting analysis to regress `imp` itself, ie whether an advertiser will win a bid. However, we figure that the provided variables are not conclusive enough to classify `imp` correctly.

In [17]:
print('Lost impressions, but clicked', ((ads['imp'] == 0) & (ads['click'] == 1)).sum())
ads[ads['imp'] == True]
ads.drop(['imp'], axis=1, inplace=True)

KeyError: 'imp'

Let's now look at the `Browser` variable.

In [19]:
ads['Browser'].describe()

count                                               3438136
unique                                               190186
top       Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1...
freq                                                 384532
Name: Browser, dtype: object

We see that the `Browser` variable captures the user agent. As such, it has too many unique values. We will aggregate these user agents into a categorical variable. There are two interesting informations in the user agent: the browser and the operating system. Let's create a column for both of them. Note that we do not concern ourselves with specific browser versions or smaller browser families for two reasons: First, the difference between specific category levels would be two small (consider `Firefox` and `Firefox Nightly`). Second, we would need to deploy libraries like `ua_parser`, which deploy either regular expressions or large lookup tables. We tried to do that and experienced that it took way too long and resulted in 70 unique values (way too large).

In [20]:
ads['Browser'] = ads['Browser'].astype(str)
def map_browser(agent):
    browsers = ['edge', 'trident', 'chrome', 'firefox', 'safari', 'opera']
    for browser in browsers:
        if browser in agent.lower():
            return 'ie' if browser == 'trident' else browser
    return 'other'


def map_os(agent):
    os_list = ['windows', 'linux', 'mac os x']
    for os in os_list:
        if os in agent.lower():
            return os
    return 'other'


ads['OS'] = ads['Browser'].map(lambda x: map_os(x), na_action=None)
ads['OS'] = ads['OS'].astype('category')
ads['Browser'] = ads['Browser'].map(lambda x: map_browser(x), na_action=None)
ads['Browser'] = ads['Browser'].astype('category')

Let's look at our new variables:

In [21]:
ads['OS'].value_counts()

windows     3364445
mac os x      51760
linux         18723
other          4403
Name: OS, dtype: int64

In [22]:
ads['Browser'].value_counts()

ie         1634893
chrome     1298420
other       409648
safari       58954
firefox      34909
opera         2507
Name: Browser, dtype: int64

Perhaps not surprisingly, most users accessed the website using Internet Explorer on a Windows operating system. The large size of `Browser == other` might seem unsettling. These are just different flavors of Chinese browsers which are obviously very specific to the Chinese market and do not generalize well.

The next information is the price which the advertiser paid for an ad. There are two columns which indicate this information. `Biddingprice` shows the bidding price of the advertiser, which was always 294 or 277 in the iPinYou competition. `Payingprice` shows the actual paid price.

In [23]:
both_na = ((ads['Payingprice'].isnull()) & (ads['Biddingprice'].isnull())).sum()
pay_na = ((ads['Payingprice'].isnull()) & ~(ads['Biddingprice'].isnull())).sum()
bid_na = (~(ads['Payingprice'].isnull()) & (ads['Biddingprice'].isnull())).sum()
print('Both missing:', both_na, 'Payingprice missing:', pay_na, 'Biddingprice missing:', bid_na)
ads['Biddingprice'].unique()

Both missing: 0 Payingprice missing: 1473696 Biddingprice missing: 0


array([ 294.,  277.])

We see that the paying price is sometimes not indicated. In these cases, `Payingprice == Biddinprice`. Thus, let's combine these columns into one.

In [24]:
ads.loc[ads['Payingprice'].isnull(), 'Payingprice'] = ads.loc[ads['Payingprice'].isnull(), 'Biddingprice']
ads['Payingprice'] = ads['Payingprice'].astype(int)
ads.drop('Biddingprice', axis=1, inplace=True)

Next up is the `AdExchange` variable. It indicates the ad exchange used for the bidding. It may not be too useful for classification, but as it is a small categorical variable, we'll include it anyway.

In [25]:
ads['AdExchange'] = ads['AdExchange'].astype('int').astype('category')
ads['AdExchange'].value_counts()

3    1422626
2     852752
1     673951
4     490002
Name: AdExchange, dtype: int64

Next we look into the `Adslotvisibility` variable. It indicates the visual importance of the ad presentation.

In [26]:
ads['Adslotvisibility'].value_counts()

Na            1710843
OtherView      715442
FirstView      597871
SecondView     174225
FourthView     123492
ThirdView      111905
FifthView        5553
Name: Adslotvisibility, dtype: int64

We see that the strings indicate six categories (FirstView--FithView, OtherView). Unfortunately, there are a lot of missing values. Nevertheless, we convert it into a categorical variable and indicate an additional level for missing values.

In [28]:
ads['Adslotvisibility'] = ads['Adslotvisibility'].astype('category')
ads['Adslotvisibility'].value_counts()

Na            1710843
OtherView      715442
FirstView      597871
SecondView     174225
FourthView     123492
ThirdView      111905
FifthView        5553
Name: Adslotvisibility, dtype: int64

We do the same with `Adslotformat`, which is very similar to `Adslotvisibility`.

In [30]:
ads['Adslotformat'] = ads['Adslotformat'].astype('category')
ads['Adslotvisibility'].value_counts()

Na            1710843
OtherView      715442
FirstView      597871
SecondView     174225
FourthView     123492
ThirdView      111905
FifthView        5553
Name: Adslotvisibility, dtype: int64

`Adslotwidth`, `Adslotheight` and `AdvertiserID` contain obvious information. Luckily, they don't contain any missing values. We'll just convert them to integers.

In [31]:
print('Missing values:', ads['AdvertiserID'].isnull().sum() + ads['Adslotwidth'].isnull().sum() + ads['Adslotheight'].isnull().sum())
int_cols = ['AdvertiserID', 'Adslotwidth', 'Adslotheight']
ads[int_cols] = ads[int_cols].astype(int)

Missing values: 0


Next, we consider the gender variables. Using two variables `male` and `female` is highly discouraged, as there is sure to be multicolinearity. Therefore, we simply delete the `Demographic_gender_male` column. We have the additional benefit of saving memory.

In [32]:
ads.drop(['Demographic_gender_male'], axis=1, inplace=True)

That concludes the basic data cleaning part. We see that we've dealt with all missing values and now enjoy the benefit of correct datatypes.

In [33]:
ads.isnull().sum()

click                              0
Browser                            0
AdExchange                         0
Adslotwidth                        0
Adslotheight                       0
Adslotvisibility                   0
Adslotformat                       0
AdvertiserID                       0
interest_news                      0
interest_education                 0
interest_automobile                0
interest_realestate                0
interest_IT                        0
interest_electronicgame            0
interest_fashion                   0
interest_entertainment             0
interest_luxury                    0
interest_homeandlifestyle          0
interest_health                    0
interest_food                      0
interest_divine                    0
interest_motherhood_parenting      0
interest_sports                    0
interest_travel_outdoors           0
interest_social                    0
Inmarket_3cproduct                 0
Inmarket_appliances                0
I

In [34]:
ads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3439331 entries, 0 to 3439330
Data columns (total 53 columns):
click                              bool
Browser                            category
AdExchange                         category
Adslotwidth                        int64
Adslotheight                       int64
Adslotvisibility                   category
Adslotformat                       category
AdvertiserID                       int64
interest_news                      bool
interest_education                 bool
interest_automobile                bool
interest_realestate                bool
interest_IT                        bool
interest_electronicgame            bool
interest_fashion                   bool
interest_entertainment             bool
interest_luxury                    bool
interest_homeandlifestyle          bool
interest_health                    bool
interest_food                      bool
interest_divine                    bool
interest_motherhood_parentin

Let's safely store and export our dataset.

In [35]:
ads.to_pickle('data/ads_clean.p')
ads.to_csv('data/ads_clean.csv', index=False)

Regression
---
Time for logistic regression.

In [1]:
from pandas import read_pickle
ads = read_pickle('data/ads_clean.p')

First, we need to get our categorical values into shape.

In [6]:
from pandas import get_dummies
browser_dummies = get_dummies(ads['Browser'], prefix='Browser')
browser_dummies.drop(['Browser_other'], axis=1, inplace=True)
adex_dummies = get_dummies(ads['AdExchange'], prefix='AdExchange')
adex_dummies.drop(['AdExchange_1'], axis=1, inplace=True)
advi_dummies = get_dummies(ads['Adslotvisibility'], prefix='Adslotvisibility')
advi_dummies.drop(['Adslotvisibility_Na'], axis=1, inplace=True)
adfo_dummies = get_dummies(ads['Adslotformat'], prefix='Adslotformat')
adfo_dummies.drop(['Adslotformat_Na'], axis=1, inplace=True)
os_dummies = get_dummies(ads['OS'], prefix='OS')
os_dummies.drop(['OS_other'], axis=1, inplace=True)
ads = ads.join([browser_dummies, adex_dummies, advi_dummies, adfo_dummies, os_dummies])
ads.drop(['Browser', 'AdExchange', 'Adslotvisibility', 'Adslotformat', 'OS'], axis=1, inplace=True)
ads.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3439331 entries, 0 to 3439330
Data columns (total 67 columns):
click                              bool
Adslotwidth                        int64
Adslotheight                       int64
AdvertiserID                       int64
interest_news                      bool
interest_education                 bool
interest_automobile                bool
interest_realestate                bool
interest_IT                        bool
interest_electronicgame            bool
interest_fashion                   bool
interest_entertainment             bool
interest_luxury                    bool
interest_homeandlifestyle          bool
interest_health                    bool
interest_food                      bool
interest_divine                    bool
interest_motherhood_parenting      bool
interest_sports                    bool
interest_travel_outdoors           bool
interest_social                    bool
Inmarket_3cproduct                 bool
Inma

Note that we dropped one level for each categorical variable, in order to avoid the dummy trap. There's only one thing to do before running the regression: Adding the intercept.

In [7]:
ads['intercept'] = 1.0

In [8]:
import statsmodels.api as sm

ads = ads.astype(int)
logit = sm.Logit(ads['click'], ads.drop(['click'], axis=1))
result = logit.fit(maxiter=100)

Optimization terminated successfully.
         Current function value: 0.002839
         Iterations 13


In [10]:
print(result.summary())

                           Logit Regression Results                           
Dep. Variable:                  click   No. Observations:              3439331
Model:                          Logit   Df Residuals:                  3439265
Method:                           MLE   Df Model:                           65
Date:                Fri, 19 Jan 2018   Pseudo R-squ.:                 0.03985
Time:                        11:42:07   Log-Likelihood:                -9764.0
converged:                       True   LL-Null:                       -10169.
                                        LLR p-value:                3.247e-129
                                      coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
Adslotwidth                         0.0006      0.000      3.331      0.001       0.000       0.001
Adslotheight                        0.0008      0.000      1.750    

In [11]:
result.mle_retvals

{'Hessian': array([[ -1.56074679e+02,  -2.57054916e+01,  -5.33302328e+02, ...,
          -2.55603010e-03,  -8.42251043e-02,  -1.98990006e-03],
        [ -2.57054916e+01,  -1.53121969e+01,  -1.58074051e+02, ...,
          -1.95926244e-03,  -2.31302088e-02,  -1.65825005e-03],
        [ -5.33302328e+02,  -1.58074051e+02,  -2.34442946e+03, ...,
          -2.01047377e-02,  -3.70828129e-01,  -1.86133988e-02],
        ..., 
        [ -2.55603010e-03,  -1.95926244e-03,  -2.01047377e-02, ...,
          -7.84655333e-06,  -7.84655333e-06,  -0.00000000e+00],
        [ -8.42251043e-02,  -2.31302088e-02,  -3.70828129e-01, ...,
          -7.84655333e-06,  -1.35382760e-04,  -0.00000000e+00],
        [ -1.98990006e-03,  -1.65825005e-03,  -1.86133988e-02, ...,
          -0.00000000e+00,  -0.00000000e+00,  -6.62485012e-06]]),
 'converged': False,
 'fopt': 0.002692530642252864,
 'iterations': 35,
 'score': array([ -2.40228815e-11,  -1.00830674e-11,  -1.25454733e-10,
          5.81040649e-12,  -2.25819028e

windows     3364445
mac os x      51760
linux         18723
other          4403
Name: OS, dtype: int64